In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array

from keras.preprocessing.text import one_hot, Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import  Embedding, LSTM
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.models import load_model
lstm_model = load_model('models/lstm_phone_review2 copy.h5')

In [3]:
# lstm_model.summary()

In [4]:
df_a=pd.read_csv("test.csv")
df2 = df_a.iloc[: , [0, 1]].copy()
# df2.head()
df2.dropna()

,Text_ID,Text
0,article_0001,Digitisation is one of the key buzzwords in th...
1,article_0002,Increase in tolerance limit up to 120 per cent...
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr..."
4,article_0005,"United Nations, Feb 4: The Serum Institute of ..."
...,...,...
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ..."
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...


In [5]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    # Remove punctuations and numbers
    sen = re.sub('[^a-zA-Z]', ' ', sen)
    sentence = sen.lower()

    

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [6]:
# import tokenizer and one hot encoder
import pickle
word_tokenizer = pickle.load(open("lstm_vec.pkl", 'rb'))
oh = pickle.load(open("ohe_encoder.pkl", 'rb'))

In [7]:
# Preprocess review text with earlier defined preprocess_text function
x=df_a.iloc[: , 1]
unseen_reviews = x

unseen_processed = []
for review in unseen_reviews:
  review = preprocess_text(review)
  unseen_processed.append(review)

In [8]:
unseen_tokenized = word_tokenizer.texts_to_sequences(unseen_processed)
maxlen=100
# Pooling instance to have maxlength of 100 tokens
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=maxlen)

In [9]:
unseen_sentiments = lstm_model.predict(unseen_padded)

125/125 [==============================] - 5s 7ms/step


In [10]:

test_class = np.argmax(unseen_sentiments, axis=1)
test_class = test_class.tolist()
# print(test_class)


In [11]:
sentiment= []

for i in test_class :
    if(i==1):
        sentiment.append(0)
    elif(i==0):
        sentiment.append(-1)
    elif(i==2):
        sentiment.append(1)

In [12]:
df_a["sentiment"]=sentiment
df_a.to_csv("lstm_output.csv")